<a href="https://colab.research.google.com/github/CoreTheGreat/HBPU-Machine-Learning-Course/blob/main/ML_Chapter4_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第四章：聚类
湖北理工学院《机器学习》课程资料

作者：李辉楚吴

笔记内容概述: 密度聚类 DBSCAN


### DBSCAN 聚类基本逻辑

数据准备

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np

# Generate virtual data (Moon data)
X_mm, y_mm = make_moons(n_samples=400, noise=0.05, random_state=42)

# Normalize X_mm using z-score
X_mm = (X_mm - np.min(X_mm, axis=0)) / (np.max(X_mm, axis=0)-np.min(X_mm, axis=0))

label_size = 18 # Label size
ticklabel_size = 14 # Tick label size

数据分析与参数选择

In [ ]:
min_pts = 3

# Comput Euclidean distance between samples
distance_map = np.zeros((X_mm.shape[0], X_mm.shape[0]))
for i in range(distance_map.shape[0]):
    for j in range(distance_map.shape[1]):
        distance_map[i,j] = np.linalg.norm(X_mm[i] - X_mm[j])

# k-Distance
k_distance = np.sort(np.sort(distance_map, axis=1)[:, min_pts])[::-1]

# Draw k-Distance figure
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(np.arange(1, len(k_distance)+1), k_distance, marker='o', linestyle='-', color='tab:blue')
ax.set_xlabel('Sample Index', fontsize=label_size)
ax.set_ylabel('k-Distance', fontsize=label_size)
ax.tick_params(axis='both', which='major', labelsize=ticklabel_size) # Set tick label size
plt.savefig(f'dbscan_kdistance.png', dpi=300)
plt.show()

确定所有Core Point

In [ ]:
r_eps = 0.04

cluster_labels = np.zeros(X_mm.shape[0]) - 1 # "-1" means noise or unlabeled

# Find all core points
snap_noise_id = 0
snap_corepoint_id = 0
for i in range(len(cluster_labels)):
    if np.sum(distance_map[i, :] <= r_eps) >= min_pts:
        cluster_labels[i] = 0 # "0" means unclustered core point

        if snap_corepoint_id < 3:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw unlabeled or noise point
            noise_idx = (cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            corepoint_idx = (cluster_labels == 0)
            circle = plt.Circle((X_mm[i, 0], X_mm[i, 1]), r_eps, edgecolor='red', facecolor='tab:red', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw core points
            ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 1: Iteration {i}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_{snap_corepoint_id+1}.png', dpi=300)
            plt.show()

            snap_corepoint_id += 1
            
    elif snap_noise_id < 3:
        fig, ax = plt.subplots(figsize=(10,10))

        # Draw unlabeled or noise point
        noise_idx = (cluster_labels == -1)
        ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

        # Draw core points
        corepoint_idx = (cluster_labels == 0)
        ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

        # Draw eps field of the current point
        circle = plt.Circle((X_mm[i, 0], X_mm[i, 1]), r_eps, edgecolor='blue', facecolor='tab:blue', alpha=0.5, zorder=1)
        ax.add_patch(circle)

        ax.set_title(f'Phase 1: Iteration {i}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
        plt.axis('off')
        # plt.savefig(f'dbscan_noise_{snap_noise_id+1}.png', dpi=300)
        plt.show()

        snap_noise_id += 1

# Cluster all core points
fig, ax = plt.subplots(figsize=(10,10))

# Draw unlabeled or noise point
noise_idx = (cluster_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw core points
corepoint_idx = (cluster_labels == 0)
ax.scatter(X_mm[corepoint_idx, 0], X_mm[corepoint_idx, 1], marker="o", c='red', s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 1: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_corepoint_final.png', dpi=300)
plt.show()

将互相可达的核心点合并成簇

In [ ]:
# Scan unlabeled core points
corepoint_cluster_labels = cluster_labels.copy()
corepoint_indices = np.where(corepoint_cluster_labels == 0)[0]

snap_corepoint_cluster_itr_idx = 0
snap_corepoint_cluster_idx = 0

cluster_id = 0 # Init cluster label

# Clustering until all core points were successfully asigned labels
while corepoint_cluster_labels[corepoint_indices].min() == 0:

    cluster_id += 1 # Build new cluster

    # Find start point, each cluster will be built completed in each loop
    for start_corepoint_idx in corepoint_indices:
        if corepoint_cluster_labels[start_corepoint_idx] == 0:
            candidate_idx = np.array([start_corepoint_idx]) # Regard the single point as a cluster
            break

    # Repeat until no unlabeled points were found
    while candidate_idx.size > 0:
        if cluster_id == 2 and snap_corepoint_cluster_idx < 1:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (corepoint_cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw unlabeled core points
            unlabel_idx = (corepoint_cluster_labels == 0)
            ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

            # Draw cluster 1 core points
            cluster1_idx = (corepoint_cluster_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (corepoint_cluster_labels == 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c='yellow', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 2: Cluster 1, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_cluster_1.png', dpi=300)
            plt.show()

            snap_corepoint_cluster_idx += 1

        # Assign cluster label to the first candidate core point
        corepoint_cluster_labels[candidate_idx[0]] = cluster_id
        circle_id = candidate_idx[0]

        # Find the neighbor core points of the first candidate core point
        neighbor_idx = np.where(distance_map[candidate_idx[0]] <= r_eps)[0] # Find neighbor points
        neighbor_idx = neighbor_idx[neighbor_idx != candidate_idx[0]] # Filter out itself
        neighbor_idx = neighbor_idx[corepoint_cluster_labels[neighbor_idx] == 0] # Remain unlabeled core points

        # Concatenate neighbor_idx to candidate_idx
        candidate_idx = np.concatenate((candidate_idx, neighbor_idx))

        # Remove the first candidate_idx core point
        candidate_idx = np.delete(candidate_idx, 0)

        # Remove duplicated points
        candidate_idx = np.unique(candidate_idx)

        if snap_corepoint_cluster_itr_idx < 3:
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (corepoint_cluster_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw unlabeled core points
            unlabel_idx = (corepoint_cluster_labels == 0)
            ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            circle = plt.Circle((X_mm[circle_id, 0], X_mm[circle_id, 1]), r_eps, edgecolor='blue', facecolor='tab:blue', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw cluster 1 core points
            cluster1_idx = (corepoint_cluster_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (corepoint_cluster_labels == 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c='yellow', s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 2: Iteration {snap_corepoint_cluster_itr_idx+1}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_corepoint_cluster_Itr{snap_corepoint_cluster_itr_idx}.png', dpi=300)
            plt.show()

            snap_corepoint_cluster_itr_idx += 1

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (corepoint_cluster_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw unlabeled core points
unlabel_idx = (corepoint_cluster_labels == 0)
ax.scatter(X_mm[unlabel_idx, 0], X_mm[unlabel_idx, 1], marker="o", c="r", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (corepoint_cluster_labels == 1)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (corepoint_cluster_labels >= 2)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=corepoint_cluster_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 2: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_corepoint_cluster.png', dpi=300)
plt.show()

将满足条件的边缘点Border Point加入簇中

In [ ]:
borderpoint_labels = corepoint_cluster_labels.copy()

# Get unlabeled points
candidate_indices = np.where(borderpoint_labels == -1)[0]

snap_border_itr = 0

# Scan candidate_idx to find border points
for candidate_idx in candidate_indices:

    # Find the neighbor points of i
    neighbor_idx = np.where(distance_map[candidate_idx] <= r_eps)[0] # Find neighbor points
    neighbor_idx = neighbor_idx[neighbor_idx != candidate_idx] # Filter out itself
    neighbor_idx = neighbor_idx[borderpoint_labels[neighbor_idx] > 0] # Remain core points

    # Candidate point is a border point
    if len(neighbor_idx) > 0:

        # Assign the closest core point's cluster label to the candicate point
        distance_to_corepoint = distance_map[candidate_idx, neighbor_idx]
        closest_corepoint_idx = neighbor_idx[np.argmin(distance_to_corepoint)]
        borderpoint_labels[candidate_idx] = borderpoint_labels[closest_corepoint_idx]

        if snap_border_itr < 3:
            # Display clustering result
            fig, ax = plt.subplots(figsize=(10,10))

            # Draw noise points
            noise_idx = (borderpoint_labels == -1)
            ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

            # Draw eps field of the current core point
            circle = plt.Circle((X_mm[candidate_idx, 0], X_mm[candidate_idx, 1]), r_eps, edgecolor='red', facecolor='tab:red', alpha=0.5, zorder=1)
            ax.add_patch(circle)

            # Draw cluster 1 core points
            cluster1_idx = (borderpoint_labels == 1)
            ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

            # Draw cluster 2 core points
            cluster2_idx = (borderpoint_labels >= 2)
            ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=borderpoint_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

            ax.set_title(f'Phase 3: Itr {snap_border_itr}, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
            plt.axis('off')
            # plt.savefig(f'dbscan_border_{snap_border_itr}.png', dpi=300)
            plt.show()

            snap_border_itr += 1

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (borderpoint_labels == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (borderpoint_labels == 1)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (borderpoint_labels >= 2)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=borderpoint_labels[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Phase 3: Final, eps = {r_eps}, minPts = {min_pts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'dbscan_border_final.png', dpi=300)
plt.show()

人工调整参数后的结果

In [ ]:
import time
from sklearn.cluster import DBSCAN

mdl_dbscan_eps = 0.05
mdl_dbscan_minpts = 3

# Define DBSCAN model
mdl_dbscan = DBSCAN(eps=mdl_dbscan_eps, min_samples=mdl_dbscan_minpts)

# Train model
start_time = time.time()
mdl_dbscan.fit(X_mm)
end_time = time.time()

print(f'Training time: {end_time - start_time:.2f} seconds')

labels_mdl_dbscan = mdl_dbscan.labels_

# Display clustering result
fig, ax = plt.subplots(figsize=(10,10))

# Draw noise points
noise_idx = (labels_mdl_dbscan == -1)
ax.scatter(X_mm[noise_idx, 0], X_mm[noise_idx, 1], marker="o", c="k", s=10**2, edgecolor="k", zorder=0)

# Draw cluster 1 core points
cluster1_idx = (labels_mdl_dbscan == 0)
ax.scatter(X_mm[cluster1_idx, 0], X_mm[cluster1_idx, 1], marker="o", c='tab:blue', s=10**2, edgecolor="k", zorder=2)

# Draw cluster 2 core points
cluster2_idx = (labels_mdl_dbscan >= 1)
ax.scatter(X_mm[cluster2_idx, 0], X_mm[cluster2_idx, 1], marker="o", c=labels_mdl_dbscan[cluster2_idx], s=10**2, edgecolor="k", zorder=2)

ax.set_title(f'Adjust parameter manually: eps = {mdl_dbscan_eps}, minPts = {mdl_dbscan_minpts}', fontsize=label_size)
plt.axis('off')
# plt.savefig(f'mdl_dbscan.png', dpi=300)
plt.show()